# Exploration et Analyse de Données de Ventes de Summit Sports  ⛷ 🧗‍♀️

## 1. Initialisation de l'environnement et chargement des données

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Ingestion des données des magasins (SS_STORES)


In [ ]:
USE ROLE sysadmin;
USE WAREHOUSE ss_de_wh;

-- LOAD SS_STORES.csv in ss_101.raw_pos.magasins

select * from ss_101.raw_pos.magasins limit 10;


## Création des tables: référence des produits, commandes et clients

Grâce au **Direct Share** de Snowflake, vous pouvez partager des données avec d'autres comptes Snowflake de manière sécurisée et instantanée, sans aucune copie physique des données ("zero copy cloning"). Cela permet un accès immédiat aux données partagées, optimisant ainsi la collaboration et la gouvernance.

In [ ]:
-- create db from share
use role accountadmin;
--CREATE OR REPLACE DATABASE sports_db FROM SHARE FKB85976.sports_db_share;

-- grant privileges to sysadmin role
GRANT IMPORTED PRIVILEGES ON DATABASE sports_db TO ROLE sysadmin;

In [ ]:
use role sysadmin;
-- products table build
-- TODO replace DB_NAME & SCHEMA_NAME
CREATE OR REPLACE TABLE ss_101.raw_pos.referentiels_produit as 
select * EXCLUDE product_url from <DB_NAME>.<SCHEMA_NAME>.SPORTS_PRODUCT_CATALOGUE;

-- orders table build 
-- TODO replace DB_NAME & SCHEMA_NAME
CREATE OR REPLACE TABLE ss_101.raw_pos.order_detail as 
select * EXCLUDE store_name from <DB_NAME>.<SCHEMA_NAME>.INSTORE_SALES_DATA_CRM3
where year(sale_date) < '2025';

-- customer loyalty table build 
-- TODO replace DB_NAME & SCHEMA_NAME
CREATE OR REPLACE TABLE ss_101.raw_customer.customer_loyalty as 
select * from <DB_NAME>.<SCHEMA_NAME>.CUSTOMERS;

Pour optimiser la performance et les coûts, nous allons ajuster la taille de notre warehouse

In [ ]:
ALTER WAREHOUSE SS_DE_WH SET WAREHOUSE_SIZE = 'X-SMALL';

## 2. Exploration des Données Agrégées et Profil de Requête


In [ ]:
//ALTER SESSION SET USE_CACHED_RESULT=FALSE;

In [ ]:
SELECT 
    c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN,
    round(COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro) / COUNT(DISTINCT isd.order_id), 0),2) AS avg_basket_size,
    COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro), 0) AS total_spend,
    COALESCE(COUNT(DISTINCT isd.order_id), 0) AS total_purchases,
    MAX(isd.sale_date) AS last_purchase_date,
    ARRAY_AGG(DISTINCT rp.product_name) WITHIN GROUP (ORDER BY rp.product_name) AS purchased_products
FROM ss_101.raw_customer.customer_loyalty c
LEFT JOIN ss_101.raw_pos.order_detail isd
    ON c.customer_id = isd.customer_id
LEFT JOIN SS_101.RAW_POS.REFERENTIELS_PRODUIT rp
    ON isd.product_id = rp.productid
where year(registration_date) > 2024 
GROUP BY c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN;

En utilisant l'historique des query de votre warehouse, consultez le query profile de votre requête pour visualiser les étapes d'exécution et identifier les éventuels bottlenecks.

## 3. Création d'une Dynamic Table harmonisée

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE ss_101.harmonized.orders_dt
    TARGET_LAG = '1 day'
    WAREHOUSE = ss_de_wh
    REFRESH_MODE = INCREMENTAL
    AS
    SELECT 
        od.ORDER_ID, od.STOREID, od.SALE_DATE, od.PRODUCT_ID, od.QUANTITY, od.SALES_PRICE_EURO, od.DISCOUNT_AMOUNT_EURO, od.PAYMENT_METHOD, od.SALES_ASSISTANT_ID, od.CUSTOMER_ID, od.CARD_ID,
        cl.FIRST_NAME, cl.LAST_NAME, 
        m.STORE_NAME, m.STORE_TYPE, m.POSTCODE,
        rp.PRODUCT_NAME, rp.BRAND, rp.MRP, rp.SALE_PRICE, rp.COLOUR, rp.PRODUCT_CATEGORY
    FROM ss_101.raw_pos.order_detail od
    LEFT JOIN ss_101.raw_customer.customer_loyalty cl ON od.customer_id = cl.customer_id
    LEFT JOIN ss_101.raw_pos.magasins m ON od.storeid = m.storeid
    LEFT JOIN ss_101.raw_pos.referentiels_produit rp ON od.product_id = rp.productid;

select * from ss_101.harmonized.orders_dt
order by sale_date desc
limit 10 ;


## Visualisation des ventes de 2024 avec Streamlit ✨

In [ ]:
ALTER WAREHOUSE SS_DE_WH SET WAREHOUSE_SIZE = 'MEDIUM';

In [ ]:
orders_df_snowpark= session.table("ss_101.harmonized.orders_dt")
df = orders_df_snowpark.to_pandas()
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
df = df[df['SALE_DATE'].dt.year == 2024]


# Statistiques clés
st.subheader("📊 Statistiques")
col1, col2, col3 = st.columns(3)
col1.metric("Total ventes (€)", f"{df['SALES_PRICE_EURO'].sum():,.2f}")
col2.metric("Quantité vendue", int(df['QUANTITY'].sum()))
col3.metric("Commandes", df['ORDER_ID'].nunique())

# Graphique des ventes par jour
st.subheader("📈 Ventes par mois")

df['SALE_MONTH'] = df['SALE_DATE'].dt.month
monthly_sales = df.groupby("SALE_MONTH")["SALES_PRICE_EURO"].sum()
fig, ax = plt.subplots()
monthly_sales.plot(ax=ax)
ax.set_ylabel("Ventes (€)")
ax.set_xlabel("Mois")

### Insert new data to inspect Dynamic Table Refresh

In [ ]:
INSERT INTO ss_101.raw_pos.order_detail(ORDER_ID, STOREID, SALE_DATE, PRODUCT_ID, QUANTITY, SALES_PRICE_EURO, DISCOUNT_AMOUNT_EURO, PAYMENT_METHOD, SALES_ASSISTANT_ID, CUSTOMER_ID, CARD_ID)
  SELECT ORDER_ID, STOREID, SALE_DATE, PRODUCT_ID, QUANTITY, SALES_PRICE_EURO, DISCOUNT_AMOUNT_EURO, PAYMENT_METHOD, SALES_ASSISTANT_ID, CUSTOMER_ID, CARD_ID 
  FROM SPORTS_DB.SPORTS_DATA.INSTORE_SALES_DATA_CRM3
  WHERE year(sale_date) = '2025';


ALTER DYNAMIC TABLE ss_101.harmonized.orders_dt REFRESH COPY SESSION;

SELECT * FROM ss_101.harmonized.orders_dt
order by sale_date desc
limit 5;

## Visualisation des ventes de 2025 avec Streamlit ✨

In [ ]:
orders_df_snowpark= session.table("ss_101.harmonized.orders_dt")
df = orders_df_snowpark.to_pandas()
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
df = df[df['SALE_DATE'].dt.year == 2025]


# Statistiques clés
st.subheader("📊 Statistiques")
col1, col2, col3 = st.columns(3)
col1.metric("Total ventes (€)", f"{df['SALES_PRICE_EURO'].sum():,.2f}")
col2.metric("Quantité vendue", int(df['QUANTITY'].sum()))
col3.metric("Commandes", df['ORDER_ID'].nunique())

# Graphique des ventes par jour
st.subheader("📈 Ventes par mois")

df['SALE_MONTH'] = df['SALE_DATE'].dt.month
monthly_sales = df.groupby("SALE_MONTH")["SALES_PRICE_EURO"].sum()
fig, ax = plt.subplots()
monthly_sales.plot(ax=ax)
ax.set_xticks(range(len(monthly_sales.index)))
ax.set_ylabel("Ventes (€)")
ax.set_xlabel("Mois")

## Création des Analytics Views

In [ ]:
CREATE OR REPLACE VIEW SS_101.HARMONIZED.customer_loyalty_metrics_v AS
SELECT 
        c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN,
        round(COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro) / COUNT(DISTINCT isd.order_id), 0),2) AS avg_basket_size,
        COALESCE(SUM(isd.sales_price_euro - isd.discount_amount_euro), 0) AS total_spend,
        COALESCE(COUNT(DISTINCT isd.order_id), 0) AS total_purchases,
        MAX(isd.sale_date) AS last_purchase_date,
        ARRAY_AGG(DISTINCT rp.product_name) WITHIN GROUP (ORDER BY rp.product_name) AS purchased_products
    FROM ss_101.raw_customer.customer_loyalty c
    LEFT JOIN ss_101.raw_pos.order_detail isd
        ON c.customer_id = isd.customer_id
    LEFT JOIN SS_101.RAW_POS.REFERENTIELS_PRODUIT rp
        ON isd.product_id = rp.productid
    GROUP BY c.CUSTOMER_ID, c.FIRST_NAME, c.LAST_NAME, c.EMAIL, c.PHONE, c.REGISTRATION_DATE, c.PREFERRED_STORE, c.MARKETING_OPT_IN; 

/*--
 • analytics view creation
--*/

-- orders_v view
CREATE OR REPLACE VIEW ss_101.analytics.orders_v
COMMENT = 'summit sports Order Detail View'
    AS
SELECT DATE(o.sale_date) AS date, * FROM ss_101.harmonized.orders_dt o;

-- customer_loyalty_metrics_v view
CREATE OR REPLACE VIEW ss_101.analytics.customer_loyalty_metrics_v
COMMENT = 'summit sports Customer Loyalty Member Metrics View'
    AS
SELECT * FROM ss_101.harmonized.customer_loyalty_metrics_v;

## Utiliser Cortex AI pour faire des analytics sur les clients

In [ ]:
CREATE OR REPLACE WAREHOUSE SS_DATASCIENCE WITH
  WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'
  WAREHOUSE_SIZE = MEDIUM
  RESOURCE_CONSTRAINT = 'MEMORY_16X_x86';

In [ ]:
use warehouse SS_DATASCIENCE;

In [ ]:
SELECT * FROM ss_101.analytics.customer_loyalty_metrics_v
WHERE AI_FILTER(PROMPT('Basé sur la liste des produits achetés, cette personne a des enfants: {0}', ARRAY_TO_STRING(purchased_products, ', ')))
limit 5;

In [ ]:
use warehouse SS_DATASCIENCE;
CREATE OR REPLACE table SS_101.HARMONIZED.customer_loyalty_metrics_t AS
    select *,
    AI_COMPLETE('mistral-large',
        CONCAT('Vous etes un assistant marketing qui travaille pour un magasin de sport. En français, écrivez un email personalisé au client <clientname>', first_name, last_name,  '</clientname> qui donne une promo personnalisé de maximum 10% sur une catégorie qui les intéresse basé sur la liste de produits achetés :  <producthistory>', ARRAY_TO_STRING(purchased_products, ', '), '</producthistory>')
        ) as product_profile
    from ss_101.analytics.customer_loyalty_metrics_v
    where avg_basket_size > 1000
    limit 3;

In [ ]:
select * from  SS_101.HARMONIZED.customer_loyalty_metrics_t;